<a href="https://colab.research.google.com/github/Amisha1019/Gen-AI-Customer-Service-Bot-Internship-Task/blob/main/Task_5_of_Gen_Ai_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python -m venv venv
!source venv/bin/activate
!pip install -U pip
!pip install streamlit scikit-learn pandas joblib nltk transformers sentencepiece

Error: Command '['/content/venv/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: line 1: venv/bin/activate: No such file or directory
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 27.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [streamlit]


In [4]:
!python train_sentiment_model.py --data customer_emotion_dataset.csv --out model.joblib

python3: can't open file '/content/train_sentiment_model.py': [Errno 2] No such file or directory


In [5]:
!streamlit run streamlit_chatbot.py --server.port 8501

Usage: streamlit run [OPTIONS] [TARGET] [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: streamlit_chatbot.py


In [6]:
import re
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
RESPONSE_TEMPLATES = {
'positive': [
"That's great to hear! How else can I assist you today?",
"I'm glad you're happy with that — anything else I can do?"
],
'negative': [
"I'm sorry you're facing that. Could you share more details so I can help?",
"I apologize for the trouble. Let's fix this together — what's the order/reference?"
],
'neutral': [
"Thanks for letting me know. How can I assist further?",
"Got it. Do you want more details on that?"
]
}

In [8]:
def simple_preprocess(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [10]:
def choose_response(sentiment, additional=None):
    template = random.choice(RESPONSE_TEMPLATES.get(sentiment, RESPONSE_TEMPLATES['neutral']))
    if additional:
        return f"{template} {additional}"
    return template

In [11]:
def load_model(path):
     return joblib.load(path)

In [12]:
def predict_with_vectorizer(model, vectorizer, texts):
    X = vectorizer.transform([simple_preprocess(t) for t in texts])
    preds = model.predict(X)
    return preds

In [14]:
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
import joblib
# from utils import simple_preprocess

In [19]:
def train(data_path, out_path):
    df = pd.read_csv('/content/customer_emotional_detecstion.csv')

    df = df.dropna(subset=['message', 'sentiment'])
    X = df['message'].astype(str).map(simple_preprocess)
    y = df['sentiment'].astype(str)


    X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.2, random_state=42, stratify=y)


    vect = TfidfVectorizer(ngram_range=(1,2), min_df=1)
    clf = LogisticRegression(max_iter=1000)
    pipeline = make_pipeline(vect, clf)


    pipeline.fit(X_train, y_train)
    preds = pipeline.predict(X_test)
    print(classification_report(y_test, preds))


    joblib.dump(pipeline, out_path)
    print('Saved model to', out_path)

In [24]:
!python your_script_name.py --data /content/customer_emotional_detecstion.csv --out sentiment_model_pipeline.joblib

python3: can't open file '/content/your_script_name.py': [Errno 2] No such file or directory


In [25]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--data', required=True)
    parser.add_argument('--out', default='model.joblib')
    args = parser.parse_args()
    train(args.data, args.out)

usage: colab_kernel_launcher.py [-h] --data DATA [--out OUT]
colab_kernel_launcher.py: error: the following arguments are required: --data
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3.12/argparse.py", line 1943, in _parse_known_args2
    namespace, args = self._parse_known_args(args, namespace, intermixed)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/argparse.py", line 2230, in _parse_known_args
    raise ArgumentError(None, _('the following arguments are required: %s') %
argparse.ArgumentError: the following arguments are required: --data

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-2055700255.py", line 5, in <cell line: 0>
    args = parser.parse_args()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/argparse.py", line 1904, in parse_args
    args, argv = self.parse_known_args(args, namespace)
  

TypeError: object of type 'NoneType' has no len()

In [27]:
import streamlit as st
import joblib
# from utils import simple_preprocess, choose_response
from transformers import pipeline
import os

In [28]:
st.set_page_config(page_title='Customer Sentiment Chatbot')


st.title('Customer Service Chatbot with Sentiment Awareness')

2025-11-05 08:56:22.619 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 08:56:22.622 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 08:56:22.720 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-11-05 08:56:22.721 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 08:56:22.722 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [30]:
use_transformer = st.sidebar.checkbox('Use transformer pipeline (no local training required)', value=False)
model_path = st.sidebar.text_input('TF-IDF model path (if using TF-IDF)', value='model.joblib')

2025-11-05 08:57:08.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 08:57:08.431 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 08:57:08.433 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 08:57:08.434 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 08:57:08.435 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 08:57:08.437 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 08:57:08.437 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 08:57:08.438 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [33]:
%%writefile streamlit_chatbot.py
import streamlit as st
import joblib
import re
import random

# Ensure simple_preprocess and choose_response are defined or imported if in a separate file
# Assuming they are defined in the notebook environment or a separate file named utils.py
# For this example, we will include them directly in the streamlit app file for simplicity.

def simple_preprocess(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

RESPONSE_TEMPLATES = {
    'positive': [
        "That's great to hear! How else can I assist you today?",
        "I'm glad you're happy with that — anything else I can do?"
    ],
    'negative': [
        "I'm sorry you're facing that. Could you share more details so I can help?",
        "I apologize for the trouble. Let's fix this together — what's the order/reference?"
    ],
    'neutral': [
        "Thanks for letting me know. How can I assist further?",
        "Got it. Do you want more details on that?"
    ]
}

def choose_response(sentiment, additional=None):
    template = random.choice(RESPONSE_TEMPLATES.get(sentiment, RESPONSE_TEMPLATES['neutral']))
    if additional:
        return f"{template} {additional}"
    return template

try:
    model_pipeline = joblib.load('sentiment_model_pipeline.joblib')
    st.success("Sentiment model loaded successfully!")
except FileNotFoundError:
    st.error("Error: 'sentiment_model_pipeline.joblib' not found. Please train the model first.")
    model_pipeline = None

st.title("Customer Sentiment Chatbot")

st.write("Enter a customer message and the chatbot will respond based on the detected sentiment.")

if model_pipeline is not None:
    user_input = st.text_input("You:", "")

    if user_input:
        sentiment_prediction = model_pipeline.predict([simple_preprocess(user_input)])[0]


        chatbot_response = choose_response(sentiment_prediction)

        st.text_area("Chatbot:", value=chatbot_response, height=100, disabled=True)

Overwriting streamlit_chatbot.py


In [37]:
sentiment_pipeline = None
tfidf_pipeline = None


if use_transformer:
    try:
        sentiment_pipeline = pipeline('sentiment-analysis')
    except Exception as e:
        st.sidebar.error('Failed to load transformer pipeline: ' + str(e))
else:
    if os.path.exists(model_path):
        try:
            tfidf_pipeline = joblib.load(model_path)
        except Exception as e:
            st.sidebar.error('Failed to load TF-IDF model: ' + str(e))


if 'history' not in st.session_state:
    st.session_state.history = []


with st.form('chat_form', clear_on_submit=True):
    user_input = st.text_input('You:')
    submitted = st.form_submit_button('Send')

2025-11-05 09:01:12.397 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 09:01:12.401 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 09:01:12.403 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 09:01:12.406 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 09:01:12.408 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 09:01:12.410 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 09:01:12.411 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 09:01:12.412 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [39]:
if submitted and user_input:
    if use_transformer and sentiment_pipeline is not None:
        res = sentiment_pipeline(user_input)[0]
        label = res.get('label', 'NEUTRAL').lower()
        if label == 'positive':
            sentiment = 'positive'
        elif label == 'negative':
            sentiment = 'negative'
        else:
            sentiment = 'neutral'
    elif tfidf_pipeline is not None:
        pred = tfidf_pipeline.predict([simple_preprocess(user_input)])[0]
        sentiment = pred
    else:
        sentiment = 'neutral'

    if sentiment == 'negative':
        bot_reply = choose_response('negative')
    elif sentiment == 'positive':
        bot_reply = choose_response('positive')
    else:
        bot_reply = choose_response('neutral')

    st.session_state.history.append(('user', user_input, sentiment))
    st.session_state.history.append(('bot', bot_reply, sentiment))

for role, text, sentiment in st.session_state.history:
    if role == 'user':
        st.markdown(f"**You** ({sentiment}): {text}")
    else:
        st.markdown(f"**Bot** ({sentiment}): {text}")


st.sidebar.markdown('---')
st.sidebar.markdown('You can train the TF-IDF model using `train_sentiment_model.py` and set its path above.')

2025-11-05 09:01:25.364 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 09:01:25.368 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 09:01:25.369 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 09:01:25.370 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 09:01:25.371 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 09:01:25.373 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-05 09:01:25.374 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator(_root_container=1, _parent=DeltaGenerator(_form_data=FormData(form_id='chat_form')))

In [ ]:
!streamlit run streamlit_chatbot.py & npx localtunnel --port 8501

⠙⠹⠸

⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.169.173.28:8501

